In [1]:
import os, sys
from tqdm import tqdm
sys.path.append(os.path.abspath(os.path.dirname(os.getcwd())))

import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt

from src.features import intended_receiver, extract_player_pos, extract_pass

# this is very useful as it makes sure that always all columns and rows of a data frame are displayed
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

<h2> Data Load </h2>

In [2]:
match1 = pd.read_csv('../metrica-data/EPV-data/match1.csv')
match2 = pd.read_csv('../metrica-data/EPV-data/match2.csv')
match3 = pd.read_csv('../metrica-data/EPV-data/match3.csv')
all_events = pd.read_csv('../metrica-data/EPV-data/all-match.csv')

In [3]:
match1.head()

,frame,session,time,phase,episode,team_poss,player_poss,event_player,event_type,A11_x,A11_y,A11_vx,A11_vy,A11_speed,A11_accel,A01_x,A01_y,A01_vx,A01_vy,A01_speed,A01_accel,A02_x,A02_y,A02_vx,A02_vy,A02_speed,A02_accel,A03_x,A03_y,A03_vx,A03_vy,A03_speed,A03_accel,A04_x,A04_y,A04_vx,A04_vy,A04_speed,A04_accel,A05_x,A05_y,A05_vx,A05_vy,A05_speed,A05_accel,A06_x,A06_y,A06_vx,A06_vy,A06_speed,A06_accel,A07_x,A07_y,A07_vx,A07_vy,A07_speed,A07_accel,A08_x,A08_y,A08_vx,A08_vy,A08_speed,A08_accel,A09_x,A09_y,A09_vx,A09_vy,A09_speed,A09_accel,A10_x,A10_y,A10_vx,A10_vy,A10_speed,A10_accel,A12_x,A12_y,A12_vx,A12_vy,A12_speed,A12_accel,A13_x,A13_y,A13_vx,A13_vy,A13_speed,A13_accel,A14_x,A14_y,A14_vx,A14_vy,A14_speed,A14_accel,B25_x,B25_y,B25_vx,B25_vy,B25_speed,B25_accel,B15_x,B15_y,B15_vx,B15_vy,B15_speed,B15_accel,B16_x,B16_y,B16_vx,B16_vy,B16_speed,B16_accel,B17_x,B17_y,B17_vx,B17_vy,B17_speed,B17_accel,B18_x,B18_y,B18_vx,B18_vy,B18_speed,B18_accel,B19_x,B19_y,B19_vx,B19_vy,B19_speed,B19_accel,B20_x,B20_y,B20_vx,B20_vy,B20_speed,B20_accel,B21_x,B21_y,B21_vx,B21_vy,B21_speed,B21_accel,B22_x,B22_y,B22_vx,B22_vy,B22_speed,B22_accel,B23_x,B23_y,B23_vx,B23_vy,B23_speed,B23_accel,B24_x,B24_y,B24_vx,B24_vy,B24_speed,B24_accel,B26_x,B26_y,B26_vx,B26_vy,B26_speed,B26_accel,B27_x,B27_y,B27_vx,B27_vy,B27_speed,B27_accel,B28_x,B28_y,B28_vx,B28_vy,B28_speed,B28_accel,ball_x,ball_y
0,1,1,0.1,1,1,B,NaN,B19,PASS,0.100440,34.731360,0.333290,-0.035174,0.335140,-0.625875,35.259840,47.031840,-0.251670,-2.273060,2.286950,-0.059494,36.397080,35.181360,-0.245772,0.820850,0.856854,-2.339236,33.401160,25.580880,0.086596,0.185292,0.204529,3.577530,34.707960,15.308640,0.906415,-2.713272,2.860670,0.463751,44.381520,52.264080,0.211816,-2.904561,2.912274,-0.290737,45.033840,34.446960,0.105120,0.959609,0.965349,-0.175408,42.255000,23.436000,1.116675,-3.071419,3.268115,0.208952,49.019040,15.245280,0.056168,-3.032358,3.032878,1.306999,56.912760,27.345600,0.183449,-5.193607,5.196846,0.145989,59.662440,31.153680,1.015902,-4.618719,4.729125,-2.593574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.731495,34.172730,-1.339970,-0.019173,1.340108,-0.864164,63.064440,14.971680,-2.051766,0.765264,2.189834,-2.652163,73.070640,33.631200,-1.938117,-1.679287,2.564430,-1.044052,72.694800,55.062720,-2.074982,-1.893343,2.808968,0.341137,44.045640,44.298000,-1.353995,-3.043234,3.330852,0.230280,49.109760,27.870480,-1.115580,-3.675937,3.841488,0.074951,60.436800,48.798000,1.122890,-4.169862,4.318407,-0.092768,59.662440,31.153680,-2.221401,-2.005754,2.992937,0.102457,54.072360,67.911840,-0.302513,0.702790,0.765133,-0.164093,47.188440,3.601440,-2.037537,-1.966954,2.832043,0.582805,40.859640,19.715760,-3.246034,1.629536,3.632097,0.325550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.476120,28.917630
1,2,1,0.2,1,1,B,NaN,B21,PASS,0.131004,34.731360,0.180384,-0.098375,0.205466,-0.538315,35.232084,47.018952,-0.193902,-2.289818,2.298013,0.291836,36.379044,35.250840,-0.276528,0.302931,0.410165,-0.777740,33.415308,25.587864,0.391235,-0.200779,0.439747,3.015208,34.709796,15.280344,0.912207,-2.746951,2.894454,-0.014227,44.360460,52.240104,0.258547,-2.868839,2.880466,-0.285167,45.030816,34.505640,-0.149472,-0.003543,0.149514,1.348017,42.264072,23.414040,1.081533,-3.113541,3.296036,0.162448,49.011264,15.162048,-0.001961,-3.164156,3.164157,0.916124,56.940408,27.356616,0.238109,-5.215412,5.220845,0.254324,59.646132,31.175424,0.724136,-4.409863,4.468922,-2.535089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.608456,34.173576,-1.191155,-0.280863,1.223820,-0.523442,63.013248,15.045912,-1.891850,0.234629,1.906344,-1.019133,73.043748,33.658920,-1.823416,-1.642523,2.454125,-0.951439,72.662940,55.100736,-2.084835,-1.968142,2.867075,0.745047,44.025228,44.280576,-1.397685,-3.069120,3.372391,0.446115,49.087080,27.905328,-1.148448,-3.674203,3.849507,-0.058290,60.453216,48.801744,1.204881,-4.125253,4.297610,-0.249277,59.646132,31

### Intended-receiver 평가 지표

In [4]:
# 각 방식에 대한 정확도 계산
methods = ['dist', 'dist and angle', 'dist and narrow angle']

# 정확도를 계산하는 함수
def calculate_accuracy(events, method):
    correct_predictions = events.apply(lambda row: int(row['Intended_Receiver'][method]['ID']) == int(row['to'][1:]), axis=1)
    accuracy = correct_predictions.mean()
    return accuracy

<h3> Metrica 1</h3>

In [5]:
df_pass_success_1 = extract_pass(all_events[all_events["game_id"] == 1], match1)
df_pass_success_1.head()

,team,start_frame,end_frame,eventName,subEventName,from,to,freeze_frame,accurate,pass_id,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd
0,Away,1,2,Pass,Pass,B19,B21,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,1,52.476120,28.917630,1,59.299344,30.117744,1
1,Away,2,7,Pass,Pass,B21,B15,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,2,59.299344,30.117744,1,61.988436,15.574464,1
2,Away,18,25,Pass,Pass,B15,B19,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,3,59.431968,13.655592,1,48.856716,21.991320,1
3,Away,31,39,Pass,Pass,B19,B21,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,4,48.902292,22.805712,1,52.426980,34.024032,1
4,Away,77,87,Pass,Pass,B21,B22,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,5,43.330140,52.546752,1,34.839396,69.503976,1


In [6]:
pass_intended_1 = intended_receiver(df_pass_success_1, match1)
pass_intended_1.head()

0it [00:00, ?it/s]

28it [00:00, 52.58it/s]c:\Users\M207\Desktop\kang\code_factory\EPV_intended\EPV\src\features.py:129: RuntimeWarning: invalid value encountered in divide
  exp_receiver_3 = np.argmax((np.amin(dist_2) /dist_2) * (np.amin(angle) / angle))
797it [00:13, 57.14it/s]


,team,start_frame,end_frame,eventName,subEventName,from,to,freeze_frame,accurate,pass_id,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd,Intended_Receiver
0,Away,1,2,Pass,Pass,B19,B21,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,1,52.476120,28.917630,1,59.299344,30.117744,1,"{'dist': {'ID': 21, 'end_x': 59.66244, 'end_y'..."
1,Away,2,7,Pass,Pass,B21,B15,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,2,59.299344,30.117744,1,61.988436,15.574464,1,"{'dist': {'ID': 15, 'end_x': 63.013248, 'end_y..."
2,Away,18,25,Pass,Pass,B15,B19,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,3,59.431968,13.655592,1,48.856716,21.991320,1,"{'dist': {'ID': 19, 'end_x': 48.329568, 'end_y..."
3,Away,31,39,Pass,Pass,B19,B21,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,4,48.902292,22.805712,1,52.426980,34.024032,1,"{'dist': {'ID': 21, 'end_x': 52.68942, 'end_y'..."
4,Away,77,87,Pass,Pass,B21,B22,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,5,43.330140,52.546752,1,34.839396,69.503976,1,"{'dist': {'ID': 22, 'end_x': 33.18483600000000..."


In [7]:
pass_intended_1.loc[2, 'Intended_Receiver']

{'dist': {'ID': 19, 'end_x': 48.329568, 'end_y': 22.850352},
 'dist and angle': {'ID': 19, 'end_x': 48.329568, 'end_y': 22.850352},
 'dist and narrow angle': {'ID': 19, 'end_x': 48.329568, 'end_y': 22.850352}}

In [8]:
accuracies = {method: calculate_accuracy(pass_intended_1, method) for method in methods}
accuracies

{'dist': 0.9611041405269761,
 'dist and angle': 0.8858218318695107,
 'dist and narrow angle': 0.8193224592220828}

<h3> Metrica 2</h3>

In [9]:
df_pass_success_2 = extract_pass(all_events[all_events["game_id"] == 2], match2)
df_pass_success_2.head()

,team,start_frame,end_frame,eventName,subEventName,from,to,freeze_frame,accurate,pass_id,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd
0,Away,21,35,Pass,Pass,B23,B20,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1746,54.078030,36.275490,1,43.991748,36.590472,1
1,Away,59,75,Pass,Pass,B20,B18,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1747,45.921816,35.845848,1,47.858904,16.206408,1
2,Away,100,114,Pass,Pass,B18,B17,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1748,49.789728,13.635576,1,33.705180,20.124144,1
3,Away,127,139,Pass,Pass,B17,B16,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1749,31.709448,22.819392,1,28.402488,41.246280,1
4,Away,158,170,Pass,Pass,B16,B15,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1750,27.789588,47.106144,1,28.991520,65.218320,1


In [10]:
pass_intended_2 = intended_receiver(df_pass_success_2, match2)
pass_intended_2.head()

10it [00:00, 45.96it/s]c:\Users\M207\Desktop\kang\code_factory\EPV_intended\EPV\src\features.py:129: RuntimeWarning: invalid value encountered in divide
  exp_receiver_3 = np.argmax((np.amin(dist_2) /dist_2) * (np.amin(angle) / angle))
964it [00:16, 58.82it/s]


,team,start_frame,end_frame,eventName,subEventName,from,to,freeze_frame,accurate,pass_id,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd,Intended_Receiver
0,Away,21,35,Pass,Pass,B23,B20,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1746,54.078030,36.275490,1,43.991748,36.590472,1,"{'dist': {'ID': 20, 'end_x': 43.247196, 'end_y..."
1,Away,59,75,Pass,Pass,B20,B18,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1747,45.921816,35.845848,1,47.858904,16.206408,1,"{'dist': {'ID': 18, 'end_x': 44.80488, 'end_y'..."
2,Away,100,114,Pass,Pass,B18,B17,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1748,49.789728,13.635576,1,33.705180,20.124144,1,"{'dist': {'ID': 17, 'end_x': 34.636248, 'end_y..."
3,Away,127,139,Pass,Pass,B17,B16,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1749,31.709448,22.819392,1,28.402488,41.246280,1,"{'dist': {'ID': 16, 'end_x': 29.97529200000000..."
4,Away,158,170,Pass,Pass,B16,B15,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1750,27.789588,47.106144,1,28.991520,65.218320,1,"{'dist': {'ID': 15, 'end_x': 31.778568, 'end_y..."


In [11]:
pass_intended_2.loc[2, 'Intended_Receiver']

{'dist': {'ID': 17, 'end_x': 34.636248, 'end_y': 21.123864},
 'dist and angle': {'ID': 17, 'end_x': 34.636248, 'end_y': 21.123864},
 'dist and narrow angle': {'ID': 17, 'end_x': 34.636248, 'end_y': 21.123864}}

In [12]:
accuracies = {method: calculate_accuracy(pass_intended_2, method) for method in methods}
accuracies

{'dist': 0.9730290456431535,
 'dist and angle': 0.9066390041493776,
 'dist and narrow angle': 0.8464730290456431}

<h3> Metrica 3</h3>

In [13]:
df_pass_success_3 = all_events[all_events["game_id"] == 3]
df_pass_success_3.replace({'team': {'Away': 'Home', 'Home': 'Away'}}, inplace=True)

df_pass_success_3 = extract_pass(df_pass_success_3, match3)
df_pass_success_3.head()

C:\Users\M207\AppData\Local\Temp\ipykernel_6132\784285754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pass_success_3.replace({'team': {'Away': 'Home', 'Home': 'Away'}}, inplace=True)


,team,start_frame,end_frame,eventName,subEventName,from,to,freeze_frame,accurate,pass_id,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd
0,Home,145,151,Pass,Pass,A10,A07,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3681,54.123390,35.081460,1,53.873208,35.068680,1
1,Home,154,171,Pass,Pass,A07,A08,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3683,53.761860,34.987320,1,68.254380,45.540792,1
2,Home,186,203,Pass,Pass,A08,A02,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3685,72.095724,43.207416,1,86.694624,29.017512,1
3,Home,212,232,Pass,Pass,A02,A03,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3687,87.316704,30.827592,1,86.325804,58.014720,1
4,Home,240,252,Pass,Pass,A03,A04,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3689,85.836240,59.311152,1,73.595952,70.548696,1


In [14]:
pass_intended_3 = intended_receiver(df_pass_success_3, match3)
pass_intended_3.head()

0it [00:00, ?it/s]c:\Users\M207\Desktop\kang\code_factory\EPV_intended\EPV\src\features.py:129: RuntimeWarning: invalid value encountered in divide
  exp_receiver_3 = np.argmax((np.amin(dist_2) /dist_2) * (np.amin(angle) / angle))
1121it [00:22, 50.43it/s]


,team,start_frame,end_frame,eventName,subEventName,from,to,freeze_frame,accurate,pass_id,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd,Intended_Receiver
0,Home,145,151,Pass,Pass,A10,A07,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3681,54.123390,35.081460,1,53.873208,35.068680,1,"{'dist': {'ID': 7, 'end_x': 54.128412, 'end_y'..."
1,Home,154,171,Pass,Pass,A07,A08,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3683,53.761860,34.987320,1,68.254380,45.540792,1,"{'dist': {'ID': 8, 'end_x': 66.790008, 'end_y'..."
2,Home,186,203,Pass,Pass,A08,A02,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3685,72.095724,43.207416,1,86.694624,29.017512,1,"{'dist': {'ID': 2, 'end_x': 84.58992, 'end_y':..."
3,Home,212,232,Pass,Pass,A02,A03,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3687,87.316704,30.827592,1,86.325804,58.014720,1,"{'dist': {'ID': 3, 'end_x': 85.101084, 'end_y'..."
4,Home,240,252,Pass,Pass,A03,A04,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3689,85.836240,59.311152,1,73.595952,70.548696,1,"{'dist': {'ID': 4, 'end_x': 73.51074, 'end_y':..."


In [15]:
pass_intended_3.loc[2, 'Intended_Receiver']

{'dist': {'ID': 2, 'end_x': 84.58992, 'end_y': 29.097144},
 'dist and angle': {'ID': 2, 'end_x': 84.58992, 'end_y': 29.097144},
 'dist and narrow angle': {'ID': 2, 'end_x': 84.58992, 'end_y': 29.097144}}

In [16]:
accuracies = {method: calculate_accuracy(pass_intended_3, method) for method in methods}
accuracies

{'dist': 0.936663693131133,
 'dist and angle': 0.8804638715432649,
 'dist and narrow angle': 0.8010704727921498}

### all_match.csv



In [18]:
all_match = pd.concat([pass_intended_1, pass_intended_2, pass_intended_3])
all_match.head()

,team,start_frame,end_frame,eventName,subEventName,from,to,freeze_frame,accurate,pass_id,xPosStart,yPosStart,episodeStart,xPosEnd,yPosEnd,episodeEnd,Intended_Receiver
0,Away,1,2,Pass,Pass,B19,B21,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,1,52.476120,28.917630,1,59.299344,30.117744,1,"{'dist': {'ID': 21, 'end_x': 59.66244, 'end_y'..."
1,Away,2,7,Pass,Pass,B21,B15,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,2,59.299344,30.117744,1,61.988436,15.574464,1,"{'dist': {'ID': 15, 'end_x': 63.013248, 'end_y..."
2,Away,18,25,Pass,Pass,B15,B19,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,3,59.431968,13.655592,1,48.856716,21.991320,1,"{'dist': {'ID': 19, 'end_x': 48.329568, 'end_y..."
3,Away,31,39,Pass,Pass,B19,B21,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,4,48.902292,22.805712,1,52.426980,34.024032,1,"{'dist': {'ID': 21, 'end_x': 52.68942, 'end_y'..."
4,Away,77,87,Pass,Pass,B21,B22,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,5,43.330140,52.546752,1,34.839396,69.503976,1,"{'dist': {'ID': 22, 'end_x': 33.18483600000000..."


In [19]:
accuracies = {method: calculate_accuracy(all_match, method) for method in methods}
accuracies

{'dist': 0.9555863983344899,
 'dist and angle': 0.8907009021512838,
 'dist and narrow angle': 0.8213046495489243}

In [21]:
len(all_match[all_match['game_id'] == 1])

KeyError: 'game_id'

In [ ]:
all_match.to_csv("../metrica")